In [1]:
import json
import pandas as pd
import sys
import sqlite3

In [2]:
print(sys.path)

['D:\\UU\\Sem3\\SDM', 'C:\\Users\\anand\\anaconda3\\python311.zip', 'C:\\Users\\anand\\anaconda3\\DLLs', 'C:\\Users\\anand\\anaconda3\\Lib', 'C:\\Users\\anand\\anaconda3', '', 'C:\\Users\\anand\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\anand\\anaconda3\\Lib\\site-packages', 'C:\\Users\\anand\\anaconda3\\Lib\\site-packages\\win32', 'C:\\Users\\anand\\anaconda3\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\anand\\anaconda3\\Lib\\site-packages\\Pythonwin']


In [8]:
sys.path.append(sys.path[0] + "\\social_data_mining\\data")

In [3]:
accounts = pd.read_csv(f"D:\\UU\\Sem3\\SDM\\social_data_mining\\data\\accounts.tsv", sep="\t")
print(accounts.head())
print("No. of accounts = ", len(accounts))

      author_id                 Type Lang Stance
0  8.508262e+06  Private individuals   fr    For
1  3.297660e+09      Advocacy actors   es    For
2  1.351437e+18  Journalistic actors   en    For
3  2.593527e+08      Advocacy actors   en    For
4  1.715861e+07      Advocacy actors   en    For
No. of accounts =  1936


In [4]:
file_path = f"D:\\UU\\Sem3\\SDM\\social_data_mining\\data\\tweets.dat"
data = []

try:
    with open(file_path, 'r') as file:
        for line in file:
            try:
                tweet = json.loads(line.strip())
                data.append(tweet)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {e}")
    tweets_df = pd.DataFrame(data)
    print(tweets_df.head())
except FileNotFoundError:
    print(f"File {file_path} not found.")
except Exception as e:
    print("Error: ", e)

                                            entities  \
0  {'mentions': [{'start': 3, 'end': 15, 'usernam...   
1  {'mentions': [{'start': 3, 'end': 19, 'usernam...   
2  {'mentions': [{'start': 3, 'end': 11, 'usernam...   
3  {'mentions': [{'start': 3, 'end': 18, 'usernam...   
4  {'mentions': [{'start': 3, 'end': 15, 'usernam...   

                                                text  possibly_sensitive  \
0  RT @MinisterTdB: Climate change won’t stop ove...               False   
1  RT @LaurenceTubiana: I just can believe it !we...               False   
2  RT @COP21en: We did it! #ParisAgreement is ado...               False   
3  RT @TheGlobalGoals: Incredible news for our wo...               False   
4  RT @StopShenhua: “The people’s resolve is such...               False   

  edit_history_tweet_ids lang                created_at  \
0   [675827469119832066]   en  2015-12-12T23:59:59.000Z   
1   [675827469006581760]   en  2015-12-12T23:59:59.000Z   
2   [675827468775718912]   en

In [5]:
print(len(tweets_df))
print(tweets_df.info())

2260916
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260916 entries, 0 to 2260915
Data columns (total 16 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   entities                object
 1   text                    object
 2   possibly_sensitive      bool  
 3   edit_history_tweet_ids  object
 4   lang                    object
 5   created_at              object
 6   referenced_tweets       object
 7   author_id               object
 8   context_annotations     object
 9   conversation_id         object
 10  id                      object
 11  public_metrics          object
 12  attachments             object
 13  in_reply_to_user_id     object
 14  geo                     object
 15  withheld                object
dtypes: bool(1), object(15)
memory usage: 260.9+ MB
None


In [6]:
from sqlite3 import Connection

In [7]:
def tweets_table(conn: Connection):
    cur = conn.cursor()
    query = """
                CREATE TABLE IF NOT EXISTS tweets (
                  id TEXT PRIMARY KEY,
                  text TEXT,
                  author_id TEXT,
                  created_at TEXT,
                  possibly_sensitive BOOLEAN,
                  lang TEXT,
                  conversation_id TEXT,
                  entities TEXT,
                  edit_history_tweet_ids TEXT,
                  context_annotations TEXT,
                  public_metrics TEXT,
                  attachments TEXT,
                  in_reply_to_user_id TEXT,
                  geo TEXT,
                  withheld TEXT
                );
            """
    cur.execute(query)

In [8]:
def reference_tweets_table(conn: Connection):
    cur = conn.cursor()
    query = """
                CREATE TABLE IF NOT EXISTS referenced_tweets (
                  tweet_id TEXT,
                  referenced_tweet_id TEXT,
                  referenced_tweet_type TEXT,
                  FOREIGN KEY (tweet_id) REFERENCES tweets(id)
                );
            """
    cur.execute(query)

In [9]:
conn = sqlite3.connect(f"D:\\UU\\Sem3\\SDM\\social_data_mining\\data\\twitter.db")
tweets_table(conn)
reference_tweets_table(conn)

In [14]:
tweet_columns = [
    "id",
    "text",
    "author_id",
    "created_at",
    "possibly_sensitive",
    "lang",
    "conversation_id",
    "entities",
    # "edit_history_tweet_ids",
    "context_annotations",
    "public_metrics",
    "attachments",
    "in_reply_to_user_id",
    "geo",
    "withheld"
]

tweet_query = f"""
    INSERT INTO tweets ({', '.join(tweet_columns)})
    VALUES ({', '.join('?' for _ in tweet_columns)})
    """

rt_query = """
    INSERT INTO referenced_tweets (tweet_id, referenced_tweet_id, referenced_tweet_type)
    VALUES (?, ?, ?)
    """

print(f"Tweet query: {tweet_query} | RT query: {rt_query}")

Tweet query: 
    INSERT INTO tweets (id, text, author_id, created_at, possibly_sensitive, lang, conversation_id, entities, context_annotations, public_metrics, attachments, in_reply_to_user_id, geo, withheld)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
     | RT query: 
    INSERT INTO referenced_tweets (tweet_id, referenced_tweet_id, referenced_tweet_type)
    VALUES (?, ?, ?)
    


In [11]:
problematic = [
    "entities",
    "context_annotations",
    "public_metrics",
    "attachments",
    "geo",
    "withheld"  
]
df = tweets_df.copy()
df[problematic] = df[problematic].astype(str)
print(df.head())

                                            entities  \
0  {'mentions': [{'start': 3, 'end': 15, 'usernam...   
1  {'mentions': [{'start': 3, 'end': 19, 'usernam...   
2  {'mentions': [{'start': 3, 'end': 11, 'usernam...   
3  {'mentions': [{'start': 3, 'end': 18, 'usernam...   
4  {'mentions': [{'start': 3, 'end': 15, 'usernam...   

                                                text  possibly_sensitive  \
0  RT @MinisterTdB: Climate change won’t stop ove...               False   
1  RT @LaurenceTubiana: I just can believe it !we...               False   
2  RT @COP21en: We did it! #ParisAgreement is ado...               False   
3  RT @TheGlobalGoals: Incredible news for our wo...               False   
4  RT @StopShenhua: “The people’s resolve is such...               False   

  edit_history_tweet_ids lang                created_at  \
0   [675827469119832066]   en  2015-12-12T23:59:59.000Z   
1   [675827469006581760]   en  2015-12-12T23:59:59.000Z   
2   [675827468775718912]   en

In [12]:
df = df.join(
    pd.DataFrame(
        df["referenced_tweets"].dropna().explode().to_list(),
    ).add_prefix("rt_")
)
print(df.head())

                                            entities  \
0  {'mentions': [{'start': 3, 'end': 15, 'usernam...   
1  {'mentions': [{'start': 3, 'end': 19, 'usernam...   
2  {'mentions': [{'start': 3, 'end': 11, 'usernam...   
3  {'mentions': [{'start': 3, 'end': 18, 'usernam...   
4  {'mentions': [{'start': 3, 'end': 15, 'usernam...   

                                                text  possibly_sensitive  \
0  RT @MinisterTdB: Climate change won’t stop ove...               False   
1  RT @LaurenceTubiana: I just can believe it !we...               False   
2  RT @COP21en: We did it! #ParisAgreement is ado...               False   
3  RT @TheGlobalGoals: Incredible news for our wo...               False   
4  RT @StopShenhua: “The people’s resolve is such...               False   

  edit_history_tweet_ids lang                created_at  \
0   [675827469119832066]   en  2015-12-12T23:59:59.000Z   
1   [675827469006581760]   en  2015-12-12T23:59:59.000Z   
2   [675827468775718912]   en

In [15]:
chunk_size=100_000
cur = conn.cursor()

for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i : i + chunk_size]
    tweets_chunk = chunk[tweet_columns].values.tolist()
    rt_chunk = chunk[["id", "rt_id", "rt_type"]].dropna().values.tolist()
    cur.executemany(tweet_query, tweets_chunk)
    cur.executemany(rt_query, rt_chunk)
    conn.commit()
    
result = cur.execute("SELECT COUNT(*) FROM tweets").fetchone()[0]
print(f"Inserted {result} tweets into the database.")
result = cur.execute("SELECT COUNT(*) FROM referenced_tweets").fetchone()[0]
print(f"Inserted {result} referenced tweets into the database.")

Inserted 2260916 tweets into the database.
Inserted 1577947 referenced tweets into the database.
